# IMAAP - Benchmark Analytics

In [1]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import edhec_risk_kit as erk
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
price_data = pd.read_excel('benchmark_data.xlsx', header=0, index_col=0, parse_dates=True)
imaap = pd.read_excel('benchmark_data.xlsx', sheet_name='IMAAP', header=0, index_col=0, parse_dates=True)

In [3]:
bmk1 = price_data[['NSE500TR', '10Y GSec Index', 'GOLD']].copy()
bmk1.dropna(inplace=True)

In [4]:
bmk2 = price_data[['NSE500TR', 'CRISIL Bond Index', 'GOLD']].copy()
bmk2.dropna(inplace=True)

In [5]:
def index_cons(securities, weights, freq='Quarterly'):
    """
    Securities = Dataframe of 3 Securities Time Series Data (Price Index)
    Weights =  List including 3 weights (float or int), adding up to 1
    Frequency = 'Quarterly' or 'Monthly'
    """
    if freq=='Quarterly':
        month1 = pd.Series(securities.index.quarter)
        month2 = pd.Series(securities.index.quarter).shift(-1)
        mask = (month1 != month2)
        bmk2q = securities[mask.values]
    elif freq =='Monthly':
        month1 = pd.Series(securities.index.month)
        month2 = pd.Series(securities.index.month).shift(-1)
        mask = (month1 != month2)
        bmk2q = securities[mask.values]

    bmk2q = bmk2q*0
    bmk2q = bmk2q + weights

    bmk2q.columns = bmk2q.columns + 'W' 
    bmk2f = securities.join(bmk2q, on='Date')
    bmk2f.iloc[0,3:] = weights
    bmk2f['Index'] = bmk2f.iloc[:,0].copy()*0
    bmk2f['Index'][0] = 10000

    bmk2f['Sec-1'] = bmk2f.iloc[:,0].copy()*0
    bmk2f['Sec-2'] = bmk2f.iloc[:,0].copy()*0
    bmk2f['Sec-3'] = bmk2f.iloc[:,0].copy()*0

    for i in range(len(bmk2f)-1):
        if bmk2f.iloc[:,3][i] > 0:
            bmk2f.iloc[:,7][i] = (bmk2f.iloc[:,6][i] * bmk2f.iloc[:,3][i])/bmk2f.iloc[:,0][i]
            bmk2f.iloc[:,8][i] = (bmk2f.iloc[:,6][i] * bmk2f.iloc[:,4][i])/bmk2f.iloc[:,1][i]
            bmk2f.iloc[:,9][i] = (bmk2f.iloc[:,6][i] * bmk2f.iloc[:,5][i])/bmk2f.iloc[:,2][i] 
        else:
            bmk2f.iloc[:,7][i] = bmk2f.iloc[:,7][i-1]
            bmk2f.iloc[:,8][i] = bmk2f.iloc[:,8][i-1]
            bmk2f.iloc[:,9][i] = bmk2f.iloc[:,9][i-1]

        bmk2f.iloc[:,6][i+1] = (bmk2f.iloc[:,7][i]*bmk2f.iloc[:,0][i+1]) + (bmk2f.iloc[:,8][i]*bmk2f.iloc[:,1][i+1]) + (bmk2f.iloc[:,9][i]*bmk2f.iloc[:,2][i+1])

    return pd.DataFrame(bmk2f[['Index']])

## IMAAP vs Benchmark I

Benchmark here consists of three components i.e.

1. NSE500 Total Return Index
2. Nifty 10-Year Benchmark G-Sec Index
3. MCX Gold


The chart and summary statistics compare the **IMAAP Portfolio** with the above stated **Benchmark** and the **CRISIL Short Term Debt Hybrid 60+40 Fund Index (CRDH6040)**

Data Source: Bloomberg

In [36]:
def index1(start, end, freq, w1, w2, w3, roll_period=3):
    bmk = index_cons(bmk1, [w1, w2, w3], freq=freq)
    bmk.columns = ['Benchmark']
    df0 = bmk.merge(imaap['IMAAP'], on='Date').merge(price_data['CRDH6040'].dropna(), on='Date')
    df0.dropna(inplace=True)
    
    #Performance
    rets = df0.pct_change().dropna()[start:end]
    rets.iloc[0,:] = 0
    df = ((1+rets).cumprod().dropna()-1)
    fig = px.line(df, x=df.index, y=df.columns)
    fig.update_layout(title = 'IMAAP Vs Benchmark I (NSE500, India 10Y GSec Index, Gold)',
                   xaxis_title='Date',
                   yaxis_title='Return (%)', font=dict(family="Segoe UI, monospace", size=13, color="#282828"),
                   legend_title_text='Portfolios', plot_bgcolor = 'White', yaxis_tickformat = '%')
    fig.update_traces(hovertemplate='Date: %{x} <br>Return: %{y:.2%}') 
    fig.update_yaxes(automargin=True)
    fig.show()
    
    #Rolling Returns & Sharpe
    roll = (1+df0.pct_change(252*roll_period).dropna()[start:end])**(1/roll_period)-1
    roll_sharpe = df0.pct_change().dropna().rolling(252*roll_period).apply(lambda x: (x.mean() / x.std()) * np.sqrt(252)).dropna()
    fig1 = px.line(roll, x=roll.index, y=roll.columns)
    fig2 = px.line(roll_sharpe, x=roll_sharpe.index, y=roll_sharpe.columns)
    fig1.update_layout(title = 'IMAAP Vs Benchmark I: '+ str(roll_period) + '-Year Annualized Rolling Returns',
                   xaxis_title='Date',
                   yaxis_title='Annualized Return (%)', font=dict(family="Segoe UI, monospace", size=13, color="#282828"),
                   legend_title_text='Portfolios', plot_bgcolor = 'White', yaxis_tickformat = '%')
    
    fig1.update_traces(hovertemplate='Date: %{x} <br>Return: %{y:.2%}') 
    fig1.update_yaxes(automargin=True)
    fig1.show()
    
    fig2.update_layout(title = 'IMAAP Vs Benchmark I: '+ str(roll_period) + '-Year Annualized Reward/Risk Ratio',
                   xaxis_title='Date',
                   yaxis_title='Reward/Risk', font=dict(family="Segoe UI, monospace", size=13, color="#282828"),
                   legend_title_text='Portfolios', plot_bgcolor = 'White',  yaxis_tickformat = '.2f')
    fig2.update_traces(hovertemplate='Date: %{x} <br>Reward/Risk: %{y:.2f}') 
    fig2.update_yaxes(automargin=True)
    fig2.show()
    
    #Summary Statistics
    summ = erk.summary_stats(rets, 0, 252).sort_values(by='Sharpe Ratio', ascending=False).T
    return summ

style = {'description_width': 'initial'}
interact(index1,
                freq = widgets.Dropdown(options=['Quarterly', 'Monthly'], value='Quarterly', style=style, description='Rebalanced: '),
                start = widgets.DatePicker(value=bmk1.index[0],style=style, description = 'Start Date: '),
                end = widgets.DatePicker(value=bmk1.index[-1],style=style, description = 'End Date: '),
                w1 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.6, description='NSE500 TR Index (%): ', style=style),
                w2 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.3, description='India 10Y GSec Index (%): ', style=style),
                w3 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.1, description='MCX Gold (%): ', style=style),
                roll_period = widgets.BoundedIntText(min= 1, max=10, step=1, value=3, description='Rolling Period (Yrs): ', style=style));

interactive(children=(DatePicker(value=Timestamp('2011-01-03 00:00:00'), description='Start Date: ', style=Des…

## IMAAP vs Benchmark II

Benchmark here consists of three components i.e.

1. NSE500 Total Return Index
2. CRISIL Composite Bond Fund Index
3. MCX Gold

The chart and summary statistics compare the **IMAAP Portfolio** with the above stated **Benchmark** and the **CRISIL Short Term Debt Hybrid 60+40 Fund Index (CRDH6040)**

Data Source: Bloomberg

In [37]:
def index2(start, end, freq, w1, w2, w3, roll_period=3):
    bmk = index_cons(bmk2, [w1, w2, w3], freq=freq)
    bmk.columns = ['Benchmark']
    df0 = bmk.merge(imaap['IMAAP'], on='Date').merge(price_data['CRDH6040'].dropna(), on='Date')
    df0.dropna(inplace=True)
    
    #Performance
    rets = df0.pct_change().dropna()[start:end]
    rets.iloc[0,:] = 0
    df = ((1+rets).cumprod().dropna()-1)
    fig = px.line(df, x=df.index, y=df.columns)
    fig.update_layout(title = 'IMAAP Vs Benchmark II (NSE500, CRISIL Composite Bond Fund Index, Gold)',
                   xaxis_title='Date',
                   yaxis_title='Return (%)', font=dict(family="Segoe UI, monospace", size=13, color="#282828"),
                   legend_title_text='Portfolios', plot_bgcolor = 'White', yaxis_tickformat = '%')
    fig.update_traces(hovertemplate='Date: %{x} <br>Return: %{y:.2%}') 
    fig.update_yaxes(automargin=True)
    fig.show()
    
    #Rolling Returns & Sharpe
    roll = (1+df0.pct_change(252*roll_period).dropna()[start:end])**(1/roll_period)-1
    roll_sharpe = df0.pct_change().dropna().rolling(252*roll_period).apply(lambda x: (x.mean() / x.std()) * np.sqrt(252)).dropna()
    fig1 = px.line(roll, x=roll.index, y=roll.columns)
    fig2 = px.line(roll_sharpe, x=roll_sharpe.index, y=roll_sharpe.columns)
    fig1.update_layout(title = 'IMAAP Vs Benchmark II: '+ str(roll_period) + '-Year Annualized Rolling Returns',
                   xaxis_title='Date',
                   yaxis_title='Annualized Return (%)', font=dict(family="Segoe UI, monospace", size=13, color="#282828"),
                   legend_title_text='Portfolios', plot_bgcolor = 'White', yaxis_tickformat = '%')
    fig1.update_traces(hovertemplate='Date: %{x} <br>Return: %{y:.2%}') 
    fig1.update_yaxes(automargin=True)
    fig1.show()
    
    fig2.update_layout(title = 'IMAAP Vs Benchmark II: '+ str(roll_period) + '-Year Annualized Reward/Risk Ratio',
                   xaxis_title='Date',
                   yaxis_title='Reward/Risk', font=dict(family="Segoe UI, monospace", size=13, color="#282828"),
                   legend_title_text='Portfolios', plot_bgcolor = 'White',  yaxis_tickformat = '.2f')
    fig2.update_traces(hovertemplate='Date: %{x} <br>Reward/Risk: %{y:.2f}') 
    fig2.update_yaxes(automargin=True)
    fig2.show()
    
    
    
    #Summary Statistics
    summ=erk.summary_stats(rets, 0, 252).sort_values(by='Sharpe Ratio', ascending=False).T
    return summ

style = {'description_width': 'initial'}
interact(index2,
                freq = widgets.Dropdown(options=['Quarterly', 'Monthly'], value='Quarterly', style=style, description='Rebalanced: '),
                start = widgets.DatePicker(value=bmk2.index[0],style=style, description = 'Start Date: '),
                end = widgets.DatePicker(value=bmk2.index[-1],style=style, description = 'End Date: '),
                w1 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.60, description='NSE500 TR Index (%): ', style=style),
                w2 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.30, description='CRISIL Composite Bond Fund Index (%): ', style=style),
                w3 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.10, description='MCX Gold (%): ', style=style),
                roll_period = widgets.BoundedIntText(min= 1, max=10, step=1, value=3, description='Rolling Period (Yrs): ', style=style));

interactive(children=(DatePicker(value=Timestamp('2004-01-01 00:00:00'), description='Start Date: ', style=Des…

## Generate Full Analytics Report

In [38]:
import webbrowser
import quantstats as qs

def show_tearsheet(option, freq, w1, w2, w3, opt):
    if option == 'Benchmark I':
        bmk = index_cons(bmk1, [w1, w2, w3], freq=freq)
        bmk.columns = ['Benchmark']
        df = bmk.merge(imaap['IMAAP'], on='Date')
        df.dropna(inplace=True)
        if opt=='HTML':
            qs.reports.html(df['IMAAP'], df['Benchmark'], output='IMAAP-BMK.html', title='IMAAP Vs Benchmark I (NSE500: 60%, India 10Y GSec Index: 30%, Gold: 10%)')
            new = 2
            url = "IMAAP-BMK.html"
            return webbrowser.open(url,new=new);
        else:
            return qs.reports.full(df['IMAAP'], df['Benchmark'])
    elif option == 'Benchmark II':
        bmk = index_cons(bmk2, [w1, w2, w3], freq=freq)
        bmk.columns = ['Benchmark']
        df = bmk.merge(imaap['IMAAP'], on='Date')
        df.dropna(inplace=True)
        if opt=='HTML':
            qs.reports.html(df['IMAAP'], df['Benchmark'], output='IMAAP-BMK.html', title='IMAAP Vs Benchmark I (NSE500: 60%, CRISIL Composite Bond Fund Index: 30%, Gold: 10%)')
            new = 2
            url = "IMAAP-BMK.html"
            return webbrowser.open(url,new=new);
        else:
            return qs.reports.full(df['IMAAP'], df['Benchmark'])

interact(show_tearsheet,
                freq = widgets.Dropdown(options=['Quarterly', 'Monthly'], value='Quarterly', style=style, description='Rebalanced: '),
                opt = widgets.Dropdown(options=['In-Page', 'HTML'], value='In-Page', style=style, description='Report: '),
                option = widgets.Dropdown(options=['Benchmark I', 'Benchmark II'], value='Benchmark I', style=style, description='Select Benchmark: '),
                w1 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.6, description='NSE500 TR Index (%): ', style=style),
                w2 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.3, description='Bond Index (%): ', style=style),
                w3 = widgets.BoundedFloatText(min= 0, max=1, step=0.01, value=0.1, description='MCX Gold (%): ', style=style));

interactive(children=(Dropdown(description='Select Benchmark: ', options=('Benchmark I', 'Benchmark II'), styl…